# 1. Set up the Huggingface Climate Policy Radar dataset.

In [ ]:
import os
import regex as re
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pgai
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from datasets import load_dataset, Features, Value
from functions import generate_embeddings_for_text
import glob

tqdm.pandas()

In [ ]:
# Login using e.g. `huggingface-cli login` in command line to access this dataset

ds = load_dataset("ClimatePolicyRadar/all-document-text-data")
ds = ds["train"]

In [ ]:
ds.features

In [ ]:
flat_ds = ds.flatten()

In [ ]:
for key in flat_ds.features.keys():
    print(type(flat_ds[18:19][key][0]))

## Save 100000 chunks in Postgres

In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv # This loads the .env file into os.environ
batch_df = pd.DataFrame(flat_ds[:100000])
# Set up the database connection using SQLAlchemy
engine = create_engine(os.getenv("DB_URL"))

# Write the pandas DataFrame to the PostgreSQL database
batch_df.to_sql('climate_policy_radar', engine, if_exists='replace', index=False)

## Attempt to change arrays to strings for entire dataset

Just keeping this here for future reference. I don't think it's needed

In [ ]:
def array_to_string(batch):
    keys_to_process = [
        "document_metadata.geographies",
        "document_metadata.languages",
        "languages",
        "pdf_data_page_metadata.dimensions",
        "text_block.coords"
    ]
    for key in keys_to_process:
        # Convert numpy arrays to lists
        batch[key] = [str(x) for x in batch[key]]
    return batch

In [ ]:
# Apply the function to the dataset. This takes the longest time
flat_ds = flat_ds.map(array_to_string, batched = True)